In [ ]:
!pip install mtcnn

In [ ]:
import numpy as np
import cv2
from mtcnn import MTCNN
#only run on google colab
from google.colab.patches import cv2_imshow

from tensorflow.keras import layers, Sequential, Input
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from keras.preprocessing.image import img_to_array
from keras.layers import Dense, Input
from tensorflow.keras.applications import InceptionV3
from keras.models import Model, Sequential

In [ ]:
#change image size as needed
img_size = 128
img_shape = (img_size, img_size, 3)

def inceptionV3():

  base_model = Sequential()
  inputs = Input((img_size, img_size, 3))
  base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=img_shape)
  x = base_model(inputs)
  x = base_model.output
  x = layers.GlobalAveragePooling2D()(x)
  x = layers.Dense(512, activation='relu')(x)
  predictions = Dense(34, activation='softmax')(x)
  model = Model(inputs=base_model.input, outputs=predictions)
  return model

new_model = inceptionV3()
new_model.summary()

In [ ]:
#change as need, according model your name
file_name = 'your-model.h5'

model_test = new_model
base_dir = '/content/drive/MyDrive/your-dir-model'

model_test.load_weights(base_dir  + file_name)

print("Loaded model from disk")

**Face Recognition Single Image Testing**

In [ ]:
detector = MTCNN()
#change as needed, e.g formate file (jpg, jpeg, png)
test_image = '/content/drive/MyDrive/your-test-image'

image = cv2.imread(test_image, cv2.COLOR_BGR2RGB)
faces = detector.detect_faces(image)

for result in faces:
    x, y, w, h = result['box']
    x1, y1 = x + w, y + h
    
    #extract face ROI, convert from BGR to RGB,
    #change image size then processing
    face = image[y:y1, x:x1]
    face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
    face = cv2.resize(face, (128, 128))
    face = img_to_array(face)
    face = preprocess_input(face)
    face = np.expand_dims(face, axis=0)    

    #read face with model
    predictions = model_test.predict(face)
    score_max = np.argmax(predictions[0])
    probabilitas = max(predictions[0] * 100)

    #class name, change as needed
    celebrity = ['Randeep Hooda', 'Rani Mukerji', 'Ranveer Singh', 'Richa Chadda', 'Riteish Deshmukh', 'Saif Ali Khan', 'Salman Khan', 'Sanjay Dutt', 'Sara Ali Khan', 'Shah Rukh Khan', 'Shahid Kapoor', 'Shilpa Shetty', 'Shraddha Kapoor', 'Shreyas Talpade', 'Shruti Haasan', 'Sidharth Malhotra', 'Sonakshi Sinha', 'Sonam Kapoor', 'Suniel Shetty', 'Sunny Deol', 'Sushant Singh Rajput', 'Taapsee Pannu', 'Tabu', 'Tamannaah Bhatia', 'Tiger Shroff', 'Tusshar Kapoor', 'Uday Chopra', 'Vaani Kapoor', 'Varun Dhawan', 'Vicky Kaushal', 'Vidya Balan', 'Vivek Oberoi', 'Yami Gautam', 'Zareen Khan']
    label = celebrity[score_max]
    print('Probabilitas: ', probabilitas)
    print('Celebrity Prediction: ', label)

    #probability of detection results 
    label_roi = "{}: {:.2f} %".format(label, probabilitas)
    
    #show results with rectangle label
    cv2.putText(image, label_roi, (x, y - 10),
    cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 255, 0), 2)
    cv2.rectangle(image, (x, y), (x1, y1), (0, 255, 0), 2)

#show output
cv2_imshow(image)
cv2.waitKey(0)

**Realtime Recognition Testing**

In [ ]:
#only use on google colab
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time

In [ ]:
#function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

#function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

In [ ]:
#JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 512, 512);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 512; // video.videoWidth;
      captureCanvas.height = 512; // video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

In [ ]:
#start run webcam
video_stream()
label_html = 'Capturing...'
bbox = ''
count = 0 
detector = MTCNN()

while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    #convert Javascript response to opencv image
    img = js_to_image(js_reply["img"])
    
    #create overlay transparent to bounding box
    bbox_array = np.zeros([512, 512, 4], dtype=np.uint8)

    faces = detector.detect_faces(img)
    faces_list = []
    preds = []

    for result in faces:
      x, y, w, h = result['box']
      x1, y1 = x + w, y + h
      
      face = image[y:y1, x:x1]
      face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
      face = cv2.resize(face, (128, 128))
      face = img_to_array(face)
      face = preprocess_input(face)
      face = np.expand_dims(face, axis=0)
     
      faces_list.append(face)
      
      if len(faces_list) > 0:
        preds = model_test.predict(faces_list)
    
      #start predicition
      for pred in preds:
        celeb_preds = pred
        score_max = np.argmax(celeb_preds[0])
        
      celebrity = ['Randeep Hooda', 'Rani Mukerji', 'Ranveer Singh', 'Richa Chadda', 'Riteish Deshmukh', 'Saif Ali Khan', 'Salman Khan', 'Sanjay Dutt', 'Sara Ali Khan', 'Shah Rukh Khan', 'Shahid Kapoor', 'Shilpa Shetty', 'Shraddha Kapoor', 'Shreyas Talpade', 'Shruti Haasan', 'Sidharth Malhotra', 'Sonakshi Sinha', 'Sonam Kapoor', 'Suniel Shetty', 'Sunny Deol', 'Sushant Singh Rajput', 'Taapsee Pannu', 'Tabu', 'Tamannaah Bhatia', 'Tiger Shroff', 'Tusshar Kapoor', 'Uday Chopra', 'Vaani Kapoor', 'Varun Dhawan', 'Vicky Kaushal', 'Vidya Balan', 'Vivek Oberoi', 'Yami Gautam', 'Zareen Khan']
      label = celebrity[score_max]
      probabilitas = max(celeb_preds * 100)

		  #probability of detection results 
      label = "{}: {:.2f}%".format(label, probabilitas)

      #show results with rectangle label
      bbox_array = cv2.putText(bbox_array, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 255, 0), 2)
      bbox_array = cv2.rectangle(bbox_array, (x, y), (x1, y1), (0, 255, 0), 2)

    bbox_array[:,:,3] = (bbox_array.max(axis=2) > 0 ).astype(int) * 255

    #convert box overlay to bytes
    bbox_bytes = bbox_to_bytes(bbox_array)
 
    #update bboc to frame, then get overlay
    bbox = bbox_bytes